# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.


## Setup Mask RCNN

In [ ]:
import os
import sys
import numpy as np
import skimage.io
from mrcnn import utils

ROOT_DIR = os.path.abspath("../../")
sys.path.append(ROOT_DIR)
sys.path.append(os.path.join(ROOT_DIR, "samples\\custom\\"))  # To find local version
import custom

MODEL_DIR = os.path.join(ROOT_DIR, "logs\\custom")
CUSTOM_MODEL_PATH = os.path.join(ROOT_DIR, "splitModels/mask_rcnn_custom_0010.h5")
if not os.path.exists(CUSTOM_MODEL_PATH):
    utils.download_trained_weights(CUSTOM_MODEL_PATH)

IMAGE_DIR = os.path.join(ROOT_DIR, "images")

## Setup Dataset

In [ ]:
import json

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the custom dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes according to the numbe of classes required to detect
        self.add_class("custom", 1, "bag")
        self.add_class("custom", 2, "belt")
        self.add_class("custom", 3, "boots")
        self.add_class("custom", 4, "footwear")
        self.add_class("custom", 5, "outer")
        self.add_class("custom", 6, "dress")
        self.add_class("custom", 7, "sunglasses")
        self.add_class("custom", 8, "pants")
        self.add_class("custom", 9, "top")
        self.add_class("custom", 10, "shorts")
        self.add_class("custom", 11, "skirt")
        self.add_class("custom", 12, "headwear")
        self.add_class("custom", 13, "scarf/tie")

        # Train or validation dataset?
        assert subset in ["train", "val", "test"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            polygons = [r['shape_attributes'] for r in a['regions'].values()]
            #labelling each class in the given image to a number

            custom = [s['region_attributes'] for s in a['regions'].values()]
            
            num_ids=[]
            #Add the classes according to the requirement
            for n in custom:
                try:
                    if n['label'] == 'bag':
                        num_ids.append(1)
                    elif n['label'] == 'belt':
                        num_ids.append(2)
                    elif n['label'] == 'boots':
                        num_ids.append(3)
                    elif n['label'] == 'footwear':
                        num_ids.append(4)
                    elif n['label'] == 'outer':
                        num_ids.append(5)
                    elif n['label'] == 'dress':
                        num_ids.append(6)
                    elif n['label'] == 'sunglasses':
                        num_ids.append(7)
                    elif n['label'] == 'pants':
                        num_ids.append(8)
                    elif n['label'] == 'top':
                        num_ids.append(9)
                    elif n['label'] == 'shorts':
                        num_ids.append(10)
                    elif n['label'] == 'skirt':
                        num_ids.append(11)
                    elif n['label'] == 'headwear':
                        num_ids.append(12)
                    elif n['label'] == 'scarf/tie':
                        num_ids.append(13)
                except:
                    pass

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "custom",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a custom dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "custom":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = image_info['num_ids']	
        #print("Here is the numID",num_ids)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        num_ids = np.array(num_ids, dtype=np.int32)	
        return mask, num_ids#.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32), 

    def load_mask_bool(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a custom dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "custom":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = image_info['num_ids']	
        #print("Here is the numID",num_ids)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.bool)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = False

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        num_ids = np.array(num_ids, dtype=np.int32)	
        return mask#.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32), 
    

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "custom":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


## Run Object


In [ ]:
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
import cv2
import os
from skimage import measure
from descartes import PolygonPatch
import alphashape
from skimage.segmentation import slic
import matplotlib.pyplot as plt
import numpy as np
from skimage.segmentation import mark_boundaries
from skimage.measure import regionprops

class SimpleConfig(Config):
    NAME = "coco_inference"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 81

class InferenceConfig(custom.CustomConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

def changeFormat(rp):
    getintoarray = []
    for i in range(rp.shape[0]):
        getintoarray.append((rp[i][1], rp[i][0]))
    return getintoarray

def segm_size(segm):
    try:
        height = segm.shape[0]
        width = segm.shape[1]
    except IndexError:
        raise

    return height, width

def merged_mask(masks):
    n= masks.shape[2]
    
    if n!=0:        
        merged_mask = np.zeros((masks.shape[0], masks.shape[1]))
        for i in range(n):
            merged_mask+=masks[...,i]
        merged_mask=np.asarray(merged_mask,dtype=np.uint8)   
        return merged_mask
    return masks[:,:,0]

def check_size(eval_segm, gt_segm):
    h_e, w_e = segm_size(eval_segm)
    h_g, w_g = segm_size(gt_segm)

    if (h_e != h_g) or (w_e != w_g):
        raise EvalSegErr("DiffDim: Different dimensions of matrices!")

class EvalSegErr(Exception):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)

def compute_acc(mask1, mask2):
    predict_mask = merged_mask(mask1)
    gt_mask = merged_mask(mask2)
    check_size(predict_mask, gt_mask)

    N_p = np.sum(gt_mask)
    N_n = np.sum(np.logical_not(gt_mask))

    TP = np.sum(np.logical_and(predict_mask, gt_mask))
    TN = np.sum(np.logical_and(np.logical_not(predict_mask), np.logical_not(gt_mask)))

    accuracy_ = ((TP + TN) / (N_p + N_n)) * 100

    return accuracy_

def compute_iou(predict_mask, gt_mask):
    if predict_mask.shape[2]==0:
        return 0
    mask1 = merged_mask(predict_mask)
    mask2 = merged_mask(gt_mask)

    # intersection = np.sum((mask1 + mask2) > 1)
    # union = np.sum((mask1 + mask2) > 0)
    # iou_score = intersection / float(union)

    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou_score = np.sum(intersection) / np.sum(union)
    #print("Iou 2 : ",iou_score)
    return iou_score * 100

config = InferenceConfig()
# config.display()

config1 = SimpleConfig()
# config1.display()

model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_weights(CUSTOM_MODEL_PATH, by_name=True)

model1 = modellib.MaskRCNN(mode="inference", config=config1, model_dir=os.getcwd())
model1.load_weights("../../splitModels/mask_rcnn_coco.h5", by_name=True)

class_names = ['BG', 'bag', 'belt', 'boots', 'footwear', 'outer', 'dress', 'sunglasses', 'pants', 'top', 'shorts', 'skirt', 'headwear', 'scraf/tie']
class_names_orang = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
                'bus', 'train', 'truck', 'boat', 'traffic light',
                'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
                'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
                'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
                'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
                'kite', 'baseball bat', 'baseball glove', 'skateboard',
                'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
                'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
                'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                'teddy bear', 'hair drier', 'toothbrush']


dataset = CustomDataset()
dataset.load_custom("datasetTesting/dataset", "test")
dataset.prepare()

IoU_list = []
acc_list = []


for image_id in dataset.image_ids:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    imageName = dataset.image_reference(image_id)

    # image = skimage.io.imread(os.path.join("datasetTesting/dataset/test/", "0000415.jpg"))
    results = model.detect([image], verbose=1)
    r = results[0]

    result1 = model1.detect([image], verbose=1)
    r1 = result1[0]

    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'], title=("./OutputFinal/clothing/" + str(image_id)))
    visualize.display_instances(image, r1['rois'], r1['masks'],r1['class_ids'], class_names_orang, r1['scores'], title=("./OutputFinal/body/" + str(image_id)))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imageSkin = cv2.imread(imageName)
    skin = cv2.cvtColor(imageSkin, cv2.COLOR_BGR2YCR_CB)

    plt.figure(figsize=(10, 10))
    _, ax = plt.subplots(1, figsize=(10,10))

    setsegmentnumber = 900
    setsigma = 2
    segments1 = slic(image, n_segments=setsegmentnumber, compactness=20, sigma=setsigma, convert2lab=True)

    flag = 1
    numberofTrue = 0
    flag_text = 1

    arrayOfClass = []
    allColorArray = []
    break_out_flag = False

    allPixelDesc = []
    allNotPredictedPixel = []
    allClassName =[]

    pixelNumber = 0
    allPixels = []

    saveCoordinates = []
    shapeCoordinates = []
    isDone = False

    saveAllPixelone = []
    saveAllPredictedMask = []*2

    AllChangedPixel = []

    color = dict(
        top=(210/255,126/255,89/255),
        footwear=(210/255,188/255,89/255),
        outer=(125/255,21/255,89/255),
        skirt=(89/255,144/255,210/255),
        dress=(102/255,204/255,0),
        pants=(0,255/255,255/255),
        belt=(255/255,0,255/255),
        headwear=(255/255,153/255,153/255), 
        boots=(255/255,0,0),
        bag=(0,1,0),
        shorts=(0,0,1),
        sunglasses=(0.3,1,0.8),
        scarf=(0.4,0.5,1),
        no_class=(0,0,0),
    )
    ClassIndex = []

    for i in r1['class_ids']:
        ClassIndex.append(class_names_orang[i])

    for region in regionprops(segments1):
        minr, minc, maxr, maxc = region.bbox
        cx, cy = region.centroid
        mask_num = len(r['class_ids'])

        #? With Body Segmentation
        # for i in range(len(ClassIndex)):
        #     if(ClassIndex[i] == "handbag"):
        #         handbagMask = r1['masks'][:,:,i]
        #         if (handbagMask[int(cx)][int(cy)] == True):
        #             for number in range(mask_num):
        #                 maskClothing = r['masks'][:,:,number]
        #                 if(maskClothing[int(cx)][int(cy)] == True
        #                     or maskClothing[int(cx-1)][int(cy)] == True
        #                         or maskClothing[int(cx+1)][int(cy)] == True
        #                             or maskClothing[int(cx)][int(cy-1)] == True
        #                                 or maskClothing[int(cx)][int(cy+1)] == True):
        #                     each_classnames = class_names[r['class_ids'][number]]
        #                     if(pixelNumber > 0):
        #                         if(allPixelDesc[pixelNumber-1][0] == region):
        #                             continue
        #                     status = "predicted"
        #                     allPixelDesc.append((region, each_classnames, str(gray[int(cx),int(cy)]),status, pixelNumber))
        #                     pixelNumber+=1
        #                     numberofTrue+=1
        #             if(numberofTrue<1):
        #                 if (handbagMask[int(cx)][int(cy)] == True):
        #                     status = "unpredicted"
        #                     allPixelDesc.append((region, "no_class", str(gray[int(cx),int(cy)]),status, pixelNumber))
        #                     pixelNumber+=1
        #             numberofTrue = 0
        #             flag+=1
        #? End With Body Segmentation

        indexPerson = ClassIndex.index("person")
        maskBody = r1['masks'][:,:,indexPerson]
        for number in range(mask_num):
            maskClothing = r['masks'][:,:,number]
            if(maskClothing[int(cx)][int(cy)] == True
                or maskClothing[int(cx-1)][int(cy)] == True
                    or maskClothing[int(cx+1)][int(cy)] == True
                        or maskClothing[int(cx)][int(cy-1)] == True
                            or maskClothing[int(cx)][int(cy+1)] == True):
                each_classnames = class_names[r['class_ids'][number]]
                if(pixelNumber > 0):
                    if(allPixelDesc[pixelNumber-1][0] == region):
                        continue
                status = "predicted"
                if(each_classnames == "scraf/tie"):
                    allPixelDesc.append((region, "scarf", str(gray[int(cx),int(cy)]),status, pixelNumber,cx,cy))
                else :
                    allPixelDesc.append((region, each_classnames, str(gray[int(cx),int(cy)]),status, pixelNumber,cx,cy))
                pixelNumber+=1
                numberofTrue+=1
        if(numberofTrue<1):
            if (maskBody[int(cx)][int(cy)] == True):
                status = "unpredicted"
                allPixelDesc.append((region, "no_class", str(gray[int(cx),int(cy)]),status, pixelNumber, cx, cy))
                pixelNumber+=1
        numberofTrue = 0
        flag+=1
        alpha = 0.5 

    print("start processing ... ")

    for i in r['class_ids']:
        allClassName.append(class_names[i])

    for i in range(len(allPixelDesc)):
        raw_coordinate = allPixelDesc[i][0].coords
        new_coordinate = changeFormat(np.array(raw_coordinate))
        cx, cy = allPixelDesc[i][0].centroid    
        lower = np.array([0, 133, 77], dtype = "uint8")
        upper = np.array([235, 173, 127], dtype = "uint8")
        skinArea = cv2.inRange(skin[int(cx)][int(cy)], lower,upper) 
        indexPerson = ClassIndex.index("person")
        maskBody = r1['masks'][:,:,indexPerson]   
        if(allPixelDesc[i][3] == "predicted"):
            if(len(saveAllPredictedMask) == 0):
                saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
                continue
            else:
                if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
                    saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
                else:
                    for j in range(len(saveAllPredictedMask)):
                        if(saveAllPredictedMask[j][0] == allPixelDesc[i][1]):
                            saveAllPredictedMask[j][1].extend(new_coordinate)
                continue

        # if(allPixelDesc[i][3] == "unpredicted"):
        #         for j in range(i,len(allPixelDesc)):
        #             if(j < len(allPixelDesc)):
        #                 if(len(allPixelDesc)!= j+1):
        #                     if(allPixelDesc[j][1] == "no_class"):
        #                         currentItem = allPixelDesc[j+1][1]
        #                     else:
        #                         break
        # if(allPixelDesc[i][3] == "unpredicted"):
        #     if allPixelDesc[i][1] == "no_class":
        #         if i == "no_class":
        #             LastcurrentItem = "no_class"
        #         else:
        #             j = i - 1
        #             while j != "no_class" and allPixelDesc[j][1] == "no_class":
        #                 j -= 1
        #             if j != "no_class":
        #                 LastcurrentItem = allPixelDesc[j][1]
        #             else:
        #                 LastcurrentItem = "no_class"
        #     else:
        #         LastcurrentItem = "no_class"

        if(allPixelDesc[i][3] == "unpredicted" and skinArea[1]==0):
            for j in range(len(allPixelDesc)):
                if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == allPixelDesc[i-1] and allPixelDesc[j][1] == allPixelDesc[i+1]):
                    allPixelDesc[i] = (allPixelDesc[i][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
                    if(allPixelDesc[i][4] not in AllChangedPixel):
                        AllChangedPixel.append(allPixelDesc[i][4])
                    if(allPixelDesc[j][1] not in np.array(saveAllPredictedMask,dtype=object)):
                        saveAllPredictedMask.append((allPixelDesc[j][1], new_coordinate))
                    else:
                        for k in range(len(saveAllPredictedMask)):
                            if(saveAllPredictedMask[k][0] == allPixelDesc[j][1]):
                                saveAllPredictedMask[k][1].extend(new_coordinate)   
                if(j < len(allPixelDesc)):
                    if(len(allPixelDesc)!= i+1):
                        if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == allPixelDesc[i+1][1]):
                            allPixelDesc[i] = (allPixelDesc[i+1][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
                            if(allPixelDesc[i][4] not in AllChangedPixel):
                                AllChangedPixel.append(allPixelDesc[i][4])
                            if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
                                saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
                            else:
                                for k in range(len(saveAllPredictedMask)):
                                    if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
                                        saveAllPredictedMask[k][1].extend(new_coordinate)
                if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == allPixelDesc[i-1][1]):
                    allPixelDesc[i] = (allPixelDesc[i-1][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
                    if(allPixelDesc[i][4] not in AllChangedPixel):
                        AllChangedPixel.append(allPixelDesc[i][4])
                    if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
                        saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
                    else:
                        for k in range(len(saveAllPredictedMask)):
                            if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
                                saveAllPredictedMask[k][1].extend(new_coordinate)
        #? backup
        # if(allPixelDesc[i][3] == "unpredicted"):
        #     for j in range(len(allPixelDesc)):
        #         if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == currentItem and allPixelDesc[i-1][3] == "predicted"):
        #             allPixelDesc[i] = (allPixelDesc[i][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
        #             if(allPixelDesc[j][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                 saveAllPredictedMask.append((allPixelDesc[j][1], new_coordinate))
        #             else:
        #                 for k in range(len(saveAllPredictedMask)):
        #                     if(saveAllPredictedMask[k][0] == allPixelDesc[j][1]):
        #                         saveAllPredictedMask[k][1].extend(new_coordinate)
        #         if(j < len(allPixelDesc)):
        #             if(len(allPixelDesc)!= i+1):
        #                 if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == currentItem and allPixelDesc[j][1] == allPixelDesc[i+1][1]):
        #                     allPixelDesc[i] = (allPixelDesc[i+1][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
        #                     if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                         saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
        #                     else:
        #                         for k in range(len(saveAllPredictedMask)):
        #                             if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
        #                                 saveAllPredictedMask[k][1].extend(new_coordinate)
        #         if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == currentItem and allPixelDesc[j][1] == allPixelDesc[i-1][1]):
        #             allPixelDesc[i] = (allPixelDesc[i-1][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
        #             if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                 saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
        #             else:
        #                 for k in range(len(saveAllPredictedMask)):
        #                     if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
        #                         saveAllPredictedMask[k][1].extend(new_coordinate)
        #? backup
        # if(allPixelDesc[i][3] == "unpredicted"):
        #     if allPixelDesc[i][1] == "no_class":
        #         if i == "no_class":
        #             LastcurrentItem = "no_class"
        #         else:
        #             j = i - 1
        #             while j != "no_class" and allPixelDesc[j][1] == "no_class":
        #                 j -= 1
        #             if j != "no_class":
        #                 LastcurrentItem = allPixelDesc[j][1]
        #             else:
        #                 LastcurrentItem = "no_class"
        #     else:
        #         LastcurrentItem = "no_class"
        #     if(allPixelDesc[i][4] == 67):
        #         print(LastcurrentItem, currentItem)

        # if(allPixelDesc[i][3] == "unpredicted"):
        #     for j in range(len(allPixelDesc)):
        #         if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == currentItem and allPixelDesc[i-1][3] == "predicted" and allPixelDesc[i+1][3] == 'predicted'):
        #             allPixelDesc[i] = (allPixelDesc[i][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
        #             if(allPixelDesc[j][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                 saveAllPredictedMask.append((allPixelDesc[j][1], new_coordinate))
        #             else:
        #                 for k in range(len(saveAllPredictedMask)):
        #                     if(saveAllPredictedMask[k][0] == allPixelDesc[j][1]):
        #                         saveAllPredictedMask[k][1].extend(new_coordinate)   
        #         if(j < len(allPixelDesc)):
        #             if(len(allPixelDesc)!= i+1):
        #                 if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == currentItem and allPixelDesc[j][1] == allPixelDesc[i+1][1]):
        #                     allPixelDesc[i] = (allPixelDesc[i+1][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
        #                     if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                         saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
        #                     else:
        #                         for k in range(len(saveAllPredictedMask)):
        #                             if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
        #                                 saveAllPredictedMask[k][1].extend(new_coordinate)
        #         if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == currentItem and allPixelDesc[j][1] == allPixelDesc[i-1][1]):
        #             allPixelDesc[i] = (allPixelDesc[i-1][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
        #             if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                 saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
        #             else:
        #                 for k in range(len(saveAllPredictedMask)):
        #                     if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
        #                         saveAllPredictedMask[k][1].extend(new_coordinate)

        # if(allPixelDesc[i][3] == "unpredicted"):
        #     cx = allPixelDesc[i][5]
        #     cy = allPixelDesc[i][6]
        #     lower = np.array([0, 48, 80], dtype = "uint8")
        #     upper = np.array([20, 255, 255], dtype = "uint8")
        #     skinArea = cv2.inRange(skin[int(cx)][int(cy)], lower,upper)
        #     if(allPixelDesc[i][4]>0):
        #         if(allPixelDesc[i-1][1] == LastcurrentItem and skinArea[1] == 0):
        #             allPixelDesc[i] = (allPixelDesc[i][0], LastcurrentItem, allPixelDesc[i][2], "predicted", allPixelDesc[i][4])
        #             if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                 saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
        #             else:
        #                 for k in range(len(saveAllPredictedMask)):
        #                     if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
        #                         saveAllPredictedMask[k][1].extend(new_coordinate)   

        # if(allPixelDesc[i][3] == "unpredicted"):
        #     cx = allPixelDesc[i][5]
        #     cy = allPixelDesc[i][6]
        #     lower = np.array([0, 48, 80], dtype = "uint8")
        #     upper = np.array([20, 255, 255], dtype = "uint8")
        #     skinArea = cv2.inRange(skin[int(cx)][int(cy)], lower,upper)
        #     if(allPixelDesc[i][4]>0):
        #         for j in range(len(allPixelDesc)):
        #             if(j < len(allPixelDesc)):
        #                 if(len(allPixelDesc)!= i+1):
        #                     if(allPixelDesc[i+1][1] == currentItem and skinArea[1] == 0):
        #                         allPixelDesc[i] = (allPixelDesc[i][0], currentItem, allPixelDesc[i][2], "predicted", allPixelDesc[i][4])
        #                         if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                             saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
        #                         else:
        #                             for k in range(len(saveAllPredictedMask)):
        #                                 if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
        #                                     saveAllPredictedMask[k][1].extend(new_coordinate)  

        # if(allPixelDesc[i][3] == "unpredicted"):
        #         for j in range(i,len(allPixelDesc)):
        #             if(j < len(allPixelDesc)):
        #                 if(len(allPixelDesc)!= j+1):
        #                     if(allPixelDesc[j][1] == "no_class"):
        #                         currentItem = allPixelDesc[j+1][1]
        #                     else:
        #                         break
        # if(allPixelDesc[i][3] == "unpredicted"):
        #     for j in range(len(allPixelDesc)):
        #         if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == currentItem and allPixelDesc[i-1][3] == "predicted" and allPixelDesc[i+1][3] == "predicted"):
        #             allPixelDesc[i] = (allPixelDesc[i][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
        #             if(allPixelDesc[j][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                 saveAllPredictedMask.append((allPixelDesc[j][1], new_coordinate))
        #             else:
        #                 for k in range(len(saveAllPredictedMask)):
        #                     if(saveAllPredictedMask[k][0] == allPixelDesc[j][1]):
        #                         saveAllPredictedMask[k][1].extend(new_coordinate)   
                # if(j < len(allPixelDesc)):
                #     if(len(allPixelDesc)!= i+1):
                #         if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == currentItem and allPixelDesc[j][1] == allPixelDesc[i+1][1]):
                #             allPixelDesc[i] = (allPixelDesc[i+1][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
                #             if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
                #                 saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
                #             else:
                #                 for k in range(len(saveAllPredictedMask)):
                #                     if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
                #                         saveAllPredictedMask[k][1].extend(new_coordinate)
                # if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == currentItem and allPixelDesc[j][1] == allPixelDesc[i-1][1]):
                #     allPixelDesc[i] = (allPixelDesc[i-1][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
                #     if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
                #         saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
                #     else:
                #         for k in range(len(saveAllPredictedMask)):
                #             if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
                #                 saveAllPredictedMask[k][1].extend(new_coordinate)
        # if(allPixelDesc[i][3] == "unpredicted"):
        #     for k in range(i,len(allPixelDesc)):
        #         if(k < len(allPixelDesc)):
        #             if(len(allPixelDesc)!= k+1):
        #                 if(allPixelDesc[k][1] == "no_class"):
        #                     currentItem = allPixelDesc[k+1][1]
        #                 if(allPixelDesc[k][1] != "no_class"):
        #                     break
        #     for j in range(len(allPixelDesc)):
        #         if(j < len(allPixelDesc)):
        #             if(len(allPixelDesc)!= i+1):
        #                 if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == currentItem and allPixelDesc[i-1][3] == "predicted" and allPixelDesc[i+1][3] == 'predicted'):
        #                     allPixelDesc[i] = (allPixelDesc[i][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
        #                     if(allPixelDesc[j][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                         saveAllPredictedMask.append((allPixelDesc[j][1], new_coordinate))
        #                     else:
        #                         for k in range(len(saveAllPredictedMask)):
        #                             if(saveAllPredictedMask[k][0] == allPixelDesc[j][1]):
        #                                 saveAllPredictedMask[k][1].extend(new_coordinate)   

        # if(allPixelDesc[i][3] == "unpredicted"):
        #     for j in range(len(allPixelDesc)):
        #         if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == allPixelDesc[i-1][1]):
        #             allPixelDesc[i] = (allPixelDesc[i-1][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
        #             if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                 saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
        #             else:
        #                 for k in range(len(saveAllPredictedMask)):
        #                     if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
        #                         saveAllPredictedMask[k][1].extend(new_coordinate)

        # if(allPixelDesc[i][3] == "unpredicted"):
        #     for j in range(len(allPixelDesc)):
        #         if(j < len(allPixelDesc)):
        #             if(len(allPixelDesc)!= i+1):
        #                 if(allPixelDesc[i][2] == allPixelDesc[j][2] and allPixelDesc[j][3] == "predicted" and allPixelDesc[j][1] == allPixelDesc[i+1][1]):
        #                     allPixelDesc[i] = (allPixelDesc[i+1][0], allPixelDesc[j][1], allPixelDesc[j][2], "predicted", allPixelDesc[i][4])
        #                     if(allPixelDesc[i][1] not in np.array(saveAllPredictedMask,dtype=object)):
        #                         saveAllPredictedMask.append((allPixelDesc[i][1], new_coordinate))
        #                     else:
        #                         for k in range(len(saveAllPredictedMask)):
        #                             if(saveAllPredictedMask[k][0] == allPixelDesc[i][1]):
        #                                 saveAllPredictedMask[k][1].extend(new_coordinate)
        #             else:
        #                 continue
    for i in range(len(AllChangedPixel)):
        if(i<len(AllChangedPixel)):
            if(len(AllChangedPixel)!= i+1):
                print("range " + str(AllChangedPixel[i]) + "-" + str(AllChangedPixel[i+1]))
                for j in range(len(allPixelDesc)):
                    if(allPixelDesc[j][3] == "unpredicted"):
                        raw_coordinate = allPixelDesc[j][0].coords
                        new_coordinate = changeFormat(np.array(raw_coordinate))
                        cx = allPixelDesc[j][5]
                        cy = allPixelDesc[j][6]
                        lower = np.array([0, 133, 77], dtype = "uint8")
                        upper = np.array([235, 173, 127], dtype = "uint8")
                        skinArea = cv2.inRange(skin[int(cx)][int(cy)], lower,upper)
                        indexPerson = ClassIndex.index("person")
                        maskBody = r1['masks'][:,:,indexPerson]
                        if(int(allPixelDesc[j][4])>int(AllChangedPixel[i]) and int(allPixelDesc[j][4]) < int(AllChangedPixel[i+1])
                        and allPixelDesc[AllChangedPixel[i]][1] == allPixelDesc[AllChangedPixel[i+1]][1] and skinArea[1]==0):
                            if(maskBody[int(cx)][int(cy)] == True):
                                if(allPixelDesc[AllChangedPixel[i]][1] not in np.array(saveAllPredictedMask,dtype=object)):
                                    saveAllPredictedMask.append((allPixelDesc[AllChangedPixel[i]][1], new_coordinate))
                                else:
                                    for k in range(len(saveAllPredictedMask)):
                                        if(saveAllPredictedMask[k][0] == allPixelDesc[AllChangedPixel[i]][1]):
                                            saveAllPredictedMask[k][1].extend(new_coordinate)     
    newArr = []
    newArr1 = []
    newArr2 = []

    for i in range(len(saveAllPredictedMask)):
        newArr2.append(saveAllPredictedMask[i][0])

    for i in range(len(saveAllPredictedMask)):
            alpha_shape = alphashape.alphashape(saveAllPredictedMask[i][1], 1)
            if(alpha_shape.type == "Polygon"):
                xx,yy = alpha_shape.exterior.coords.xy
                xs = [min(xx),max(xx),max(xx),min(xx),min(xx)]
                ys = [min(yy),min(yy),max(yy),max(yy),min(yy)]
                ax.plot(xs, ys, color=color[saveAllPredictedMask[i][0]],linestyle='dashed')
            if(alpha_shape.type == "MultiPolygon"):
                for j in range(len(list(alpha_shape))):
                    xx, yy = alpha_shape[j].exterior.coords.xy
                    xs = [min(xx),max(xx),max(xx),min(xx),min(xx)]
                    ys = [min(yy),min(yy),max(yy),max(yy),min(yy)]
                    ax.plot(xs, ys, color=color[saveAllPredictedMask[i][0]], linestyle='dashed')
            ax.add_patch(PolygonPatch(alpha_shape, alpha=0.5,color=color[saveAllPredictedMask[i][0]],ec='black'))

    print(allPixelDesc)

    plt.imshow(image.astype(np.uint8))
    plt.imshow(mark_boundaries((image).astype(np.uint8), segments1, color=(0.2,0.3,0.2)))
    plt.axis('off') 
    plt.axis("image") 
    plt.savefig("./OutputFinal/superpixel/" + str(image_id) + ".jpg",  bbox_inches='tight') 

    newMaskEvaluation = []*2
    superPixelmask = np.zeros([600, 400, len(saveAllPredictedMask)],
                        dtype=np.uint8)
    for i in range(len(saveAllPredictedMask)):
        getOuterOnly = alphashape.alphashape(saveAllPredictedMask[i][1], 0.8)
        if(getOuterOnly.type == "Polygon"):
            xx,yy = getOuterOnly.exterior.coords.xy
            rr, cc = skimage.draw.polygon(yy, xx)
            superPixelmask[rr, cc, i] = 1
        if(getOuterOnly.type == "MultiPolygon"):
            for j in range(len(list(getOuterOnly))):
                xx, yy = getOuterOnly[j].exterior.coords.xy
                rr, cc = skimage.draw.polygon(yy, xx)
                superPixelmask[rr, cc, i] = 1

    if(superPixelmask.size != 0):
        IoU = compute_iou(mask, r['masks'])
        acc= compute_acc(mask, r['masks'])

        IoUSP = compute_iou(mask, superPixelmask)
        accSP= compute_acc(mask, superPixelmask)
    else:
        IoU = 0
        acc= 0

        IoUSP = 0
        accSP= 0

    # IoUSPRcnn = compute_iou(r['masks'], superPixelmask)
    # accSPRcnn = compute_acc(r['masks'], superPixelmask)

    for i in class_ids:
        if(class_names[i] not in newArr):
            newArr.append(class_names[i])

    for i in r['class_ids']:
        if(class_names[i] not in newArr1):
            newArr1.append(class_names[i])

    newArr.sort()
    newArr1.sort()
    newArr2.sort()

    words = ["Image Id : " + str(image_id), 
            "Path File : " + str(imageName),
            "Hasil IoU Mask R-CNN : " + str(IoU), 
            "Hasil Acc Mask R-CNN: " + str(acc),
            "Hasil IoU Superpixel: " + str(IoUSP),
            "Hasil Acc Superpixel: " + str(accSP),
            "Ground Truth Label: " + str(newArr),
            "Predicted Label Using Mask R-CNN: " + str(newArr1),
            "Predicted Label Using Superpixel: " + str(newArr2),]
    with open('./OutputFinal/output.txt', 'a') as f:
        for word in words:
            f.write(word)
            f.write("\n")

    print("Image Name : ", imageName)
    
    print('Hasil IoU Mask R-CNN:',IoU)
    print('Hasil Acc Mask R-CNN:',acc)

    print('Hasil IoU Superpixel:',IoUSP)
    print('Hasil Acc Superpixel:',accSP)

